## Join USWTDB Turbine Locations to PLUSWIND locations
This allows us to get windspeed and power generation for the USWTDB turbines.

Install python packages using pip install -r requirements.txt

Copy the files pluswind-plant-list-20221213.csv and uswtdb_v5.csv into this folder.

In [ ]:
import pandas as pd
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt

In [ ]:
# Preview uswtdb
uswtb_preview = pd.read_csv("uswtdb_v5.csv", nrows=10)
uswtb_preview

In [ ]:
df_uswtdb = pd.read_csv("uswtdb_v5.csv")

geoms = [shapely.Point(xy) for xy in zip(df_uswtdb["xlong"], df_uswtdb["ylat"])]
uswtdb = gpd.GeoDataFrame(df_uswtdb, geometry=geoms)
uswtdb.set_crs(epsg=4326, inplace=True)

In [ ]:
m_uswtdb = uswtdb.explore(
    # column="BoroName", # Column to use for color styling
    # tooltip=["BoroName", "Shape_Area"], # Columns to display in the tooltip
    popup=True, # Enable popups on click
    tiles="CartoDB positron", # Use a different tile set
    cmap="viridis" # Colormap
)

m_uswtdb

In [ ]:
# Pluswind Preview
pluswind_preview = pd.read_csv("pluswind-plant-list-20221213.csv", nrows=10)
pluswind_preview

In [ ]:
df_pluswind = pd.read_csv("pluswind-plant-list-20221213.csv")

geoms = [shapely.Point(xy) for xy in zip(df_pluswind["lon"], df_pluswind["lat"])]
pluswind = gpd.GeoDataFrame(df_pluswind, geometry=geoms)
pluswind.set_crs(epsg=4326, inplace=True)


In [ ]:
m = pluswind.explore(
    # column="BoroName", # Column to use for color styling
    # tooltip=["BoroName", "Shape_Area"], # Columns to display in the tooltip
    popup=True, # Enable popups on click
    # tiles="CartoDB positron", # Use a different tile set
    tiles="OpenStreetMap",
    cmap="viridis" # Colormap
)

m

In [ ]:
# Join on the nearest

uswtdb.sjoin_nearest(pluswind[["EIA_ID", "geometry"]], how="inner", distance_col="proximity", max_distance=0.015)

In [ ]:
# Above has too many records (11826). Try merging the other direction

joined = pluswind[["EIA_ID", "geometry"]].sjoin_nearest(uswtdb, how="inner", distance_col="proximity", max_distance=0.015)
joined.rename(columns={"EIA_ID": "pluswind_eia_id"}, inplace=True)
joined

In [ ]:
joined.to_csv("pluswind-joined-to-uswtdb.csv")